RNN

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 샘플 텍스트 데이터
data = """나는 오늘 기분이 좋아.
나는 내일도 기분이 좋을 거야.
기분이 좋은 날엔 춤을 추고 싶어."""

In [3]:
# 1. 토큰화: 텍스트 데이터를 숫자로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.split('\n'))
sequences = tokenizer.texts_to_sequences(data.split('\n'))

In [4]:
# 2. 단어 인덱스 확인
word_index = tokenizer.word_index
print("단어 인덱스:", word_index)

단어 인덱스: {'기분이': 1, '나는': 2, '오늘': 3, '좋아': 4, '내일도': 5, '좋을': 6, '거야': 7, '좋은': 8, '날엔': 9, '춤을': 10, '추고': 11, '싶어': 12}


In [5]:
# 3. 시퀀스를 학습 데이터로 변환
input_sequences = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        input_sequences.append(sequence[:i+1])

In [6]:
print(input_sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 5], [2, 5, 1], [2, 5, 1, 6], [2, 5, 1, 6, 7], [1, 8], [1, 8, 9], [1, 8, 9, 10], [1, 8, 9, 10, 11], [1, 8, 9, 10, 11, 12]]


In [7]:
# 4. 패딩 처리
max_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_len,padding='pre')

In [8]:
# 5. 입력(X)과 출력(y) 분리
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [9]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  0  0  0  2]
 [ 0  0  0  2  5]
 [ 0  0  2  5  1]
 [ 0  2  5  1  6]
 [ 0  0  0  0  1]
 [ 0  0  0  1  8]
 [ 0  0  1  8  9]
 [ 0  1  8  9 10]
 [ 1  8  9 10 11]]


In [10]:
print(y)

[ 3  1  4  5  1  6  7  8  9 10 11 12]


In [11]:
# 6. 출력(y)을 원-핫 인코딩
y = tf.keras.utils.to_categorical(y, num_classes=len(word_index) + 1)

In [12]:
# RNN 모델 정의
rnn_model = Sequential([
    Embedding(input_dim=len(word_index) + 1,
              output_dim=10,
              input_length=max_len - 1), # 임베딩 층
    SimpleRNN(64, return_sequences=False), # RNN 층
    Dense(len(word_index) + 1,
          activation='softmax') # 출력 층
])

# 모델 컴파일 및 학습
rnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
rnn_model.fit(X, y, epochs=20, verbose=1)


Epoch 1/20


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0833 - loss: 2.5623
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0833 - loss: 2.5485
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0833 - loss: 2.5347
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1667 - loss: 2.5208
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3333 - loss: 2.5065
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3333 - loss: 2.4918
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3333 - loss: 2.4765
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3333 - loss: 2.4606
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.3333 - loss: 2.4438
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.2500 - loss: 2.4264
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2500 - loss: 2.4082
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.2500 - loss: 2.3894
Epoch 13/20
1/

In [13]:
# LSTM 모델 정의
lstm_model = Sequential([
    Embedding(input_dim=len(word_index) + 1,
              output_dim=10,
              input_length=max_len - 1), # 임베딩 층
    LSTM(64, return_sequences=False), # LSTM 층
    Dense(len(word_index) + 1,
          activation='softmax') # 출력 층
])
# 모델 컴파일 및 학습
lstm_model.compile(loss='categorical_crossentropy', 
                   optimizer='adam', 
                   metrics=['accuracy'])
lstm_model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0833 - loss: 2.5655
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1667 - loss: 2.5635
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0833 - loss: 2.5614
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0833 - loss: 2.5593
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1667 - loss: 2.5572
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.1667 - loss: 2.5551
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1667 - loss: 2.5528
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.1667 - loss: 2.5505
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.1667 - loss: 2.5481
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1667 - loss: 2.5456
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.1667 - loss: 2.5429
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1667 - loss: 2.5401
Epo

In [14]:
# GRU 모델 정의
gru_model = Sequential([
    Embedding(input_dim=len(word_index) + 1,output_dim=10,input_length=max_len - 1), # 임베딩 층
    GRU(64, return_sequences=False), # GRU 층
    Dense(len(word_index) + 1,activation='softmax') # 출력 층
])
# 모델 컴파일 및 학습
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20


c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.5674
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0833 - loss: 2.5638
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1667 - loss: 2.5602
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.2500 - loss: 2.5566
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.2500 - loss: 2.5530
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1667 - loss: 2.5493
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1667 - loss: 2.5455
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1667 - loss: 2.5415
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1667 - loss: 2.5375
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.1667 - loss: 2.5332
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.1667 - loss: 2.5288
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1667 - loss: 2.5241
Epoch 13/2

In [19]:
def generate_text(model, tokenizer, seed_text, next_words, max_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break
    return seed_text

In [20]:
# 텍스트 생성 예제
seed_text = "나는 내일"
print("RNN 생성 결과:", generate_text(rnn_model, tokenizer, seed_text, next_words=5, max_len=max_len))
print("LSTM 생성 결과:", generate_text(lstm_model, tokenizer, seed_text, next_words=5, max_len=max_len))
print("GRU 생성 결과:", generate_text(gru_model, tokenizer, seed_text, next_words=5, max_len=max_len))

RNN 생성 결과: 나는 내일 기분이 기분이 기분이 기분이 기분이
LSTM 생성 결과: 나는 내일 기분이 기분이 기분이 기분이 기분이
GRU 생성 결과: 나는 내일 기분이 기분이 기분이 기분이 기분이
